## classification
* 지도학습-회귀/분류
* 비지도학습-군집화/차원축소

* 예측한다는건 y = wx+b에서 w와 b를 찾는 것.
* 우선 hypothesis를 정함. 임의의 wx+b를 정한다. 
* 첫번째 hypothesis에 대한 첫번째 cost를 확인. 예측값과 실제값의 차를 err. 비용함수, 손실함수.


* 머신러닝은 weight의 경사가 0일때 그만둠.
* opyimization중 문제가 local minimun과 overfitting이 있음.
overfitting이 좋다고 생각할 수 있는데 지금 있는 데이터가 너무 잘 맞아서 새로운 데이터에 대한 대응이 안된다. 일반화가 덜된 상황임. p40에 보면 overfiiting에서 새로운 초록색이 왔을 때 잘못 분류할 가능성이 크다.

* x수에 비해 데이터가 작을 때, 
* cost함수는 우리가 원하는 방향으로 학습 시키기 위한 것. 
* Trainset vs Testset(8:2)로 진행. trainset을 많이시키면 기본 데이터에 너무 많아서 testset이 오히려 안맞음(새로운 데이터에대한 대응이 안됨)
* 교차검증- trainset을 학습시키고 각 단계마다 평균을 냄.


## 텐서플로우 실행하기


In [1]:
import sys 
!{sys.executable} -m pip install tensorflow

In [2]:
import tensorflow as tf

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


* tf.constant하면 안나오는데 sess를 만들어서 그 안에 넣어야 값이 나옴

In [3]:
hello = tf.constant('Hello TensorFlow!')
print(hello)

sess = tf.Session()
result = sess.run(hello)
print(result)
sess.close()

Tensor("Const:0", shape=(), dtype=string)
b'Hello TensorFlow!'


## 계산그래프 만들기
 * tensor를 만드는 건 그냥 노드를 만드는 것일 뿐 
 * run을 해야 흘러감
 * 데이터타입이 꼭 맞아야지 계산이 됨

In [4]:
import tensorflow as tf

node1 = tf.constant(3, tf.float32)#explicitly dtype = tf.floot32
node2 = tf.constant( 4.0) #implicitly tf.float32
node3 = tf.add(node1, node2)


In [5]:
print("node1:", node1)
print("node2:", node2)
print("node3:", node3)

node1: Tensor("Const_1:0", shape=(), dtype=float32)
node2: Tensor("Const_2:0", shape=(), dtype=float32)
node3: Tensor("Add:0", shape=(), dtype=float32)


In [6]:
session = tf.Session()
print("session.run([node1,node2]):",session.run([node1,node2]))
print("session.run([node3]):",session.run(node3))

session.close() #session을 열었으면 닫아줘야함.


session.run([node1,node2]): [3.0, 4.0]
session.run([node3]): 7.0


### 빌딩구조와 실행구조의 분리
* 장점 : 1. Back propagation (Gradient 자동계산)이 쉽다.
        2. session을 동시에 돌리면 되기때문에 병렬구성이 쉽다.
        3. 딥러닝은 데이터가 많으므로 시간이 오래걸림. 분산처리에 능통함.
        딥러닝 구상하기가 쉬움. 
        4. 그래프로 확인가능하니까 구조파악하기가 쉬움
* 단점 : 디버깅이 어렵다.디버깅 해봐야 어디가 오류난지 알 수 있음. 

### 자료형

* 상수형 - shape(행렬)
shape이 없는건 스칼라


In [7]:
#다른 type으로
a = tf.constant(1, dtype = tf.float32)
print(a)
print(a.shape) #스칼라형태로 출력됨
with tf.Session() as session:
    print(session.run(a))

Tensor("Const_3:0", shape=(), dtype=float32)
()
1.0


In [8]:
#다른 shape으로
a = tf.constant(1, dtype = tf.float32, shape=(1,))
print(a)
print(a.shape) #벡터형태
with tf.Session() as session:
    print(session.run(a))

Tensor("Const_4:0", shape=(1,), dtype=float32)
(1,)
[1.]


In [9]:
#2차원으로 설정
a = tf.constant(1, dtype = tf.float32, shape=(1,1))
print(a)
print(a.shape) #행렬형태
with tf.Session() as session:
    print(session.run(a))

Tensor("Const_5:0", shape=(1, 1), dtype=float32)
(1, 1)
[[1.]]


In [10]:
#3차원으로 설정
a = tf.constant(1, dtype = tf.float32, shape=(1,1,1))
print(a)
print(a.shape) #다행렬
with tf.Session() as session:
    print(session.run(a))

Tensor("Const_6:0", shape=(1, 1, 1), dtype=float32)
(1, 1, 1)
[[[1.]]]


* placeholder - 값을 실행시키는 단계에서 feed_dict 인자를 통해 전달./값을 갈아끼울 수 있음


In [11]:
input_data = [1,2,3]
x = tf.placeholder(dtype = tf.float32) #그래프를 만드는 것. 
#x값이 float일거야. 라고만 정의한 것 값을 지정하지 않음. 

y = x*2
print("x: ",x)
print("y: ",y)

session = tf.Session()

#feed_dict를 통해 값을 전달합니다 ( 이 때 x에 값이 들어가게 됨.형변환이 일어남.)
#실행할 때 [1,2,3] float형으로 받기 때문에 [2.0 4.0 6.0]으로 받음 0은 생략됨.
#피쳐값에 x는 새로운 데이터가 오면 갈아끼워야하기때문에 placeholder를 씀. 
result = session.run(y, feed_dict = {x:input_data})
print(result)
session.close()

x:  Tensor("Placeholder:0", dtype=float32)
y:  Tensor("mul:0", dtype=float32)
[2. 4. 6.]


* variable은 변할 수 있는 값. weight는 update 시켜야하기때문에(낮은값으로)

* 주로 weight에서 사용함

In [12]:
x = tf.placeholder(dtype = tf.float32)
x_data = [1 ,2, 4]

#초기값: tf.random_normal([1,3],stddev = 0.35)
#
weights = tf.Variable([3], dtype = tf.float32)
biases = tf.Variable([4], dtype = tf.float32)

y = weights*x + biases

print("x: ", x)
print("weights: " , weights)
print("biases: " , biases)

x:  Tensor("Placeholder_1:0", dtype=float32)
weights:  <tf.Variable 'Variable:0' shape=(1,) dtype=float32_ref>
biases:  <tf.Variable 'Variable_1:0' shape=(1,) dtype=float32_ref>


### 개별 initializer

In [13]:
with tf.Session() as session:
    #초기화 오퍼레이션을 실행
    session.run([weights.initializer, biases.initializer])
    
    result = session.run(y, feed_dict = {x:x_data})
    print(result)
    
    #변수변경 //변수 재 할당
    session.run(tf.assign(weights,[2]))
    session.run(tf.assign(biases,[5]))
    
    #다시실행
    result = session.run(y, feed_dict ={x:x_data})
    print(result)

[ 7. 10. 16.]
[ 7.  9. 13.]


## Global_variable

In [14]:
#변수 초기화 작업
init_op = tf.global_variables_initializer()

with tf.Session() as session:
    #초기화 오퍼레이션을 실행
    session.run(init_op)
    
    result = session.run(y , feed_dict = {x: x_data})
    print(result)
    
    #변수 변경 // 변수 재할당 
    session.run(tf.assign(weights,[2]))
   
    
    #다시실행
    result = session.run(y, feed_dict ={x:x_data})
    print(result)

[ 7. 10. 16.]
[ 6.  8. 12.]


In [15]:
a = tf.constant(1) # 0차원 스칼라
b = tf.constant([1,2],dtype = tf.float32) #1차원 벡터
c = tf.constant([[1,2],[3,4]],dtype = tf.float32) #2차원 행렬
d = tf.constant([[[1.0,2.0],[3,4]],[[5,6],[7,8]]]) # 3차원


print('a: ' ,a)
print('b: ' ,b)
print('c: ' ,c)
print('d: ' ,d)



a:  Tensor("Const_7:0", shape=(), dtype=int32)
b:  Tensor("Const_8:0", shape=(2,), dtype=float32)
c:  Tensor("Const_9:0", shape=(2, 2), dtype=float32)
d:  Tensor("Const_10:0", shape=(2, 2, 2), dtype=float32)


## Tensor의 속성(Rank, Shape, Types)

### Rank

In [25]:
print("a's rank:" , a._rank())
print("a's rank:" , b._rank())
print("a's rank:" , c._rank())
print("a's rank:" , d._rank())

print("*" *50)

print("a's rank:" , tf.rank(a))
print("a's rank:" , tf.rank(b))
print("a's rank:" , tf.rank(c))
print("a's rank:" , tf.rank(d))


a's rank: 0
a's rank: 1
a's rank: 2
a's rank: 3
**************************************************
a's rank: Tensor("Rank_13:0", shape=(), dtype=int32)
a's rank: Tensor("Rank_14:0", shape=(), dtype=int32)
a's rank: Tensor("Rank_15:0", shape=(), dtype=int32)
a's rank: Tensor("Rank_16:0", shape=(), dtype=int32)


### Shape

In [ ]:
m,n = c.get_shape() #언패킹

In [24]:
print("a's shape:" , a.get_shape())
#print("a's shape:" , a.shape)와 같은 결과
print("b's shape:" , b.get_shape())
print("c's shape:" , c.get_shape())
print("d's shape:" , d.get_shape())

print("*" *50)

print("a's rank:" , tf.shape(a))
print("b's rank:" , tf.rank(b))
print("c's rank:" , tf.rank(c))
print("d's rank:" , tf.rank(d))


a's shape: ()
b's shape: (2,)
c's shape: (2, 2)
d's shape: (2, 2, 2)
**************************************************
a's rank: Tensor("Shape_2:0", shape=(0,), dtype=int32)
b's rank: Tensor("Rank_10:0", shape=(), dtype=int32)
c's rank: Tensor("Rank_11:0", shape=(), dtype=int32)
d's rank: Tensor("Rank_12:0", shape=(), dtype=int32)


### dtpye
* data type을 통용시켜야하고 범위가 큰 쪽으로 맞춰진다.(캐스팅 형 변환)

In [22]:
print("a's type: " , a.dtype)
print("b's type: " , b.dtype)
print("c's type: " , c.dtype)
print("d's type: " , d.dtype)


a's type:  <dtype: 'int32'>
b's type:  <dtype: 'float32'>
c's type:  <dtype: 'float32'>
d's type:  <dtype: 'float32'>


## Session관리

## tf.get_default_session()
 * 디폴트 되어있는 세션
 * with로 해야 session이 반환됨. 

In [26]:
session = tf.get_default_session()
print(session)

None


In [27]:
with tf.Session() as sess: 
     #with절로만 했을 때 get_default_session쓸 수 있음 
    print(tf.get_default_session())

* initializer - 따로하거나 global로 진행하거나
* session해주고 거기서 run해줘야 진행함. 
* with에서 쓸 수 있다는건 b.initializer과 같다는 말. = sess.run(b.initializer)
* 디폴트세션은 with절 안에서만 진행하기때문에 initializer을 하면 아무것도 안뜸. get_default_sessionn이 없으면

* sess라는 객체를 만들어서 접근하지 않고 변수에서 바로 접근하기 위해서는(.initializer이나 .eval) with절이나 interactiveSession을 써야한다는 것. 
* 

## Graph 관리하기

* get_default_graph() : 디폴트 값을 가져오면 어떤 객체값이 가져와짐. 
* is는 같냐는걸 물어보는거.
* a.graph

* 텐서 하나가 노드임. 
* a = tf.Variable하면 a라는 텐서를 하나 만드는 것임. 텐서가 어느 그래프에 속해있냐는게 a.graph임. 
* graph 객체는 캔버스라고 생각하자 (get_dafault하면 캔버스 호출하는거)
* graph를 두개로 만드는 거는 한가지를 모델링해서 짠하고 나오는게 아님. 모델링 하나하나가 그래프임. 그래서 직선그래프 하나, 선형 그래프 하나, 그래서 그래프끼리 비교함. 
* 디폴트 그래프를 a라고 가정하면 그 전에 아무 그래프를 안만들었으니까 a.graph가 디폴트 그래프임. 
* 그리고 graph를 하나 만들어서 디폴트로 만들어 준것 그 안에서 b 텐서를 만들어 준거고 b.graph가 graph냐라고 하면 ㅇㅋ
